# MeqSilhouette v0.7 demo

Roger Deane, February 2018

This is a demo/tutorial of the significantly refactored version of MeqSilhouette, an effort that was motivated by the desire to empower the user. MeqSilhouette can now be run in several different ways other than from source, including Docker and in a Jupyter Notebook. The plots and explanations below will demonstrate the usability and some new features. **Two of the key changes** in the refactored version are (1) each simulation returns a class instance, which has all relevant simulation outputs as attributes - making it far more accessible to the new user and an expert user working on new capabilities; (2) given the above, simulations with a change in input parameter need not be rerun in their entirety - only the relevant functions need be recalled. Both of these aspects should greatly simplify and economise statistical investigations of specific effects like atmospheric conditions and antenna pointing error models. 

## <font color='#336699'>0. Running MeqSilhouette v0.7</font>

As described in the GitHub repository README file, MeqSilhouette can be run in a number of different ways. Most recently, I've added the ability to run this within a Juypter Notebook. This is powerful approach to learning how to use the software, testing new ideas, or writing a demo/tutorial. For completeness, I include all methods below. 

The software can be run in four primary modes:

### a) Through the terminal (Ubuntu 16.04 or 14.04 recommended)
```shell
$ python driver/run_meqsilhouette.py input/eht230.json
```

### b) In a Juypter Notebook

Start up notebook, then enter:

```python
from run_meqsilhouette import *
config = '/path/to/config.json'
sim = run_meqsilhouette(config)
```

### c) In a Docker container

While setting up the required enviroment to run MeqSilhouette is just a few step process (for Ubuntu 14.04, 16.04),
one can avoid system dependencies entirely with Docker. See the GitHub repo instructions for setting this up. 

### d) In a virtual environment

Similarly, a virtual enviroment may provide the solution you need. See README for setup instructions. 

### MeqSilhouette v0.7 capabilties

For the busy reader, I've decided to start by showing a few basic output plots that quickly illustrate some of  MeqSilhouette v0.7 capabilties. The following plots are for the following observational setup:
- 24 hour observation with the EHT2017 array towards Sgr A\*
- observing frequency = 230 GHz 
- bandwidth = 4 GHz
- number of frequency channels = 512 (~8 MHz channel width)
- troposphere above each station (turbulence phase, mean opacity, delay, sky noise included)
- antenna pointing constant offset for each 10 min scan 

Further details of these input parameters will be given later on, but for now I show the output plots. 

In [1]:
from IPython.display import HTML, display

## <font color='#336699'>1. Basic observation overview plots</font>

$uv$-coverage plot colour-coded by baseline. The annuli are in 1 G$\lambda$ intervals. Note, JCMT is ommitted given overlap with SMA. 
<img src="plots/uv-coverage_legend.png">

The $uv$-tracks can be colour-coded by any time-variable parameter, of course. Here, for example, I show them colour-coded by each baseline's mean and minimum antenna elevation (i.e. mean and minimum of the two antennas in each baseline). This may be useful in generating a range of diagnostics and exploring systematics.  

| mean baseline elevation | minimum baseline elevation | 
|:---:|:---:|
| [![](plots/uv-coverage_colorize_mean_elevation.png)]() | [![](plots/uv-coverage_colorize_min_elevation.png)]() |







Elevation versus time for each station.
<img src="plots/antenna_elevation_vs_time.png">

Number of visbiltiies per $uv$-distance bin. 
<img src="plots/num_vis_perbin.png">

And the corresponding sensitivity per $uv$-distance bin. Note this includes thermal and sky noise, from non-zero opacities. 
<img src="plots/sensitivity_perbin.png">

## <font color='#336699'>2. Antenna Pointing Error Plots</font>

In this model, I assume a fixed offset pointing error per scan. Each scan is 10 minutes long in this synthetic dataset, and the offsets are determined by a station-specific value drawn from a Gaussian distribution of user-defined width. 

Here I plot the angular offset as a function of time for all EHT stations. Note that no data are present for antennas below their user-defined elevation limit. 
<img src="plots/pointing_angular_offset_vs_time.png">

Here I illustrate what those angular offsets translate into in terms of amplitude errors, given that each stations has it's own unique primary beam width. Here I assume typical pointing error offsets less than the primary beam FWHM, enabling the use of a Gaussian primary beam shape. However, more complex beam shape can easily be inserted. 
<img src="plots/pointing_amp_error_vs_angular_offset.png">

This translates into an unkown station-dependent amplitude error as a function of time. 
<img src="plots/pointing_amp_error_vs_time.png">

## <font color='#336699'>3. The Tropospheric module</font>
This comprises of a mean and turbulent compnent. I start with an illustration of simluated turbulent phase above each station due to Kolmogorov turbulence. Note that phases do not wrap around -180,180 degrees for easier viewing. 

<img src="plots/turbulent_phase_vs_time.png">



The mean component is determined by the station-defined PWV. The only reason for the time-variability at present is due to the airmass (elevation) effects being incorporated. 

<img src="plots/mean_delay_vs_time.png">


Of course, the non-zero PWV also has a (frequency-dependent) sky noise contribution, as illustrated below. Note that AATM within MeqSilhouette include a full radiation transfer, including treatment of pressure-broadened lines. 

<img src="plots/zenith_skytemp_vs_freq.png">

This also results in amplitude loss due to the frequency-dependent non-unity transmission:

<img src="plots/zenith_transmission_vs_freq.png">

The above plots are the zenith values, however, all are time-dependent (or rather, elevation dependent). Below, I show heatmaps to demonstrate the time dependance of transmission, per station. 

<img src="plots/transmission_vs_freq_ALMA.png">
<img src="plots/transmission_vs_freq_JCMT.png">
<img src="plots/transmission_vs_freq_LMT.png">
<img src="plots/transmission_vs_freq_PdBI.png">
<img src="plots/transmission_vs_freq_PV.png">
<img src="plots/transmission_vs_freq_SMA.png">
<img src="plots/transmission_vs_freq_SMT.png">
<img src="plots/transmission_vs_freq_SPT.png">


Similary, one can show the turbulent phase frequency and time denpendence, however, this is far less pronounced. I just show the frequency-dependent turbulent phase for ALMA below. 




<img src="plots/turbulent_phase_waterfall_plot_ALMA.png">


## <font color='#336699'>4. Interstellar Scattering</font>

As part of the effort to simplify MeqSilhouette and make it more user-friendly, I decided remove the Scatterbrane interface. All analyses that wish to include ISM scattering should input images with scattering already applied.  

## <font color='#336699'>5. Time-variable sky model</font>

If the input sky model is a FITS image, a static sky model is assumed. However, if a time-variable sky model is desired, simply set the input sky model to the path of the directory containing the input FITS images. Note, this assumes that 
a) the input sky models are ordered by filename,
b) sky models spaced at equal time intervals 
c) each image time interval = obslength / number_of_sky_models (get right variable name)
d) ...

There is a some basic error handling, however, it's best to double-check the output to ensure it's behaving as expected. A more complex user-interface for time variability was found to be unwieldy, however, suggestions are welcome. 

**Important: The input FITS image must be the form of the example image included (in *MeqSilhouette/input/skymodels/central_point_src_1.4Jy.fits*), simply modify your input images to be in this form in order for it to be parsed to MeqTrees successfully. **



## <font color='#336699'>6. Input Bandpasses</font>

Iniyan is currently implementing the ability to insert antenna bandpasses. This will give the option to provide a text file (eventually a CASA calibration table) that is interpolated based on the user's input correlator configuration. The first implementation of this will be time-invariant bandpasses with amplitude-only interpolation (i.e. bandpass phase assumed zero). 

## <font color='#336699'>7. Input configuration file</font>

The input config file (a .json file) has the following format (but does not include the inserted comments). Many of the parameters have boolean input (i.e. **0** or **1** to disable/enable). We're working on a format that is easier on the eye. 

```json
"outdirname":"output/EHT230_4chan_all_test",                         # output directory name
"input_fitsimage":"input/sky_models/central_point_src_1.4Jy.fits",   # input sky model (FITS format, ignores header, only uses data and input parmeters in this .json file)
"input_fitsimage_cellsize": "0.1e-6arcsec",                          # input sky model pixel size 
"output_to_logfile":0,                                               # loses colour output
"add_thermal_noise":1,                                               # based on SEFDs
"make_image":0,                                                      # dirty image useing wsclean. Selection of an imager may be include in the future. 
"exportuvfits":0,                                                    # using CASA task `exportuvfits'
"station_info":"input/station_info/eht_station_info.txt",            # includes each station's weather, pointing error rms, primary beam info
"elevation_limit":0.174,                         # limit is applied to all antennas at this stage, will be changed in future
"ms_antenna_table":"input/antenna_tables/EHT",   # CASA antenna table, order checked to correspond with station_info
"ms_datacolumn":"DATA",                          # output column
"ms_RA":266.416837,       # note input skymodel FITS image coordinates are ignored, only ms_RA, ms_DEC are used
"ms_DEC":-29.000781,
"ms_nu":230,              # centre observing frequency, units: GHz
"ms_dnu":4,               # total bandwidth, units: GHz
"ms_nchan":10,            # number of channels
"ms_obslength":24,        # total observation length, units: hours
"ms_tint":100,            # correlator dump time, units: seconds
"ms_StartTime":"UTC,2017/04/01/00:00:00.00", # UTC start time, CASA simobserve can lead to a few second offset from this
"ms_nscan":1,             # to split observation length into individual scans - see simms documentation
"ms_scan_lag":0,          # see simms documentation
"ms_makeplots": 1,        # make basic overview plots, as seen in Section #1 of this demo
"im_cellsize":"3e-6arcsec",   # output image pixel size
"im_npix":64,             # number of pixels in output image
"im_stokes":"I",          # Stokes parameter of output image
"im_weight":"uniform",    # imaging weighting scheme to be used
"trop_enabled":1,         # use troposphere module? 
"trop_wetonly":0,         # only include wet delay, opacity
"trop_attenuate":1,       # apply opacity-induced amplitude loss
"trop_noise":1,           # add sky-noise from non-zero opacity
"trop_turbulence":1,      # enable Kolmogorov turbulence, coherence times are specified in station_info file
"trop_mean_delay":1,      # add mean delay from PWV specified in station_info file
"trop_percentage_calibration_error":100,   # apply a percentage correction to all phase errors (turbulent and mean)
"trop_fixdelays":0,       # used for testing CASA fringe-fitter: set delays to be constant in time
"trop_fixdelay_max_picosec": 0, # specify range of random fixed delay, units: pico-seconds, minimum = 0
"trop_makeplots": 1,      #  make output plots of all troposphere-related effects
"pointing_enabled":1,     # use pointing error module? 
"pointing_time_per_mispoint": 10,   # model assume constant pointing offset for this interval, units: minutes
"pointing_makeplots": 1   # make output plots for all pointing error related effects
```

The above .json file points to two addtional input files. The first is the station information file. This includes the following inforation in a simple text file. 

1. Station name
2. SEFD, units: Jy
3. PWV, units: mm
4. ground pressure, units: millibar
5. ground temperature, units: Kelvin
6. coherence time, units: seconds
7. pointing error rms, units: arcseconds
8. primary beam @ 230 GHz, units: arcseconds
9. primary beam model (Gaussian, Consine^3, etc. Only Gaussian implemented right now)

|station |     sefd [Jy]    |   pwv [mm]   |    gpress [mb]  |    gtemp [K]   |    c_time [sec] |  ptg_rms [arcsec]  |   PB_FWHM230 [arcsec]  |    PB_model |
| ---    |   ---       |  ---    |  ---      |   ---     |  ---     | ---       |    ---       |     ---         |
|SMA     |    6000.0   |  0.7    | 625       |    280    |      30  |   1.0     |     44.7     |     gaussian    |
|SMT     |    1300.0   |  4.0    |  520      |    273    |      10  |    1.0    |      26.8    |      gaussian   |
|LMT     |    560.0    |  4.0    |  600      |    273    |      30  |    1.0    |      5.36    |      gaussian   |
|ALMA    |    220.0    |  0.9    |  550      |    265    |      60  |   1.0     |     22.35    |     gaussian    |
|PV      |    2000.0   |  1.6    |  723      |    273    |      20  |    1.0    |      8.9     |      gaussian   |
|PdBI    |    1600.0   |  2      |  600      |    270    |      25  |   1.0     |     17.8     |     gaussian    |
|JCMT    |    5000.0   |  1      |  680      |    270    |      30  |    1.0    |      15      |      gaussian   |
|SPT     |    1600.0   |  0.57   |  600      |    270    |      40  |    1.0    |      10      |      gaussian   |
|APEX    |    4500.0   |  1.6    |  680      |    223    |      60  |    1.0    |      10      |      gaussian   |


Other inputs include a CASA style antenna table. This is in the standard CASA format, with an example included in the git repo. There is also a script included in the repo (*MeqSilhouette/framework/add_ant.py*), should you wish to add additional stations. 